In [454]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA

In [455]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [456]:
df = pd.read_csv('../data/raw/WildBlueberryPollinationSimulationData.csv')

In [457]:
df.drop('Row#', axis= 1, inplace = True)

In [458]:
df.shape

(777, 17)

Trabajaremos sobre el skew de HoneyBee

In [459]:
from scipy.stats import boxcox

In [460]:
df['honeybee'] = df['honeybee'] + 1

In [461]:
df['honeybeeBC'], l = boxcox(df['honeybee'])

In [462]:
df.head()

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield,honeybeeBC
0,37.5,1.75,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.410652,0.408159,31.678898,3813.165795,0.301481
1,37.5,1.75,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,1.0,0.10,0.444254,0.425458,33.449385,4947.605663,0.301481
2,37.5,1.75,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,16.0,0.26,0.383787,0.399172,30.546306,3866.798965,0.301481
3,37.5,1.75,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,1.0,0.10,0.407564,0.408789,31.562586,4303.943030,0.301481
4,37.5,1.75,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.354413,0.382703,28.873714,3436.493543,0.301481


In [463]:
df.drop(columns=['honeybee'], axis = 1 , inplace = True)

Ahora trabajaremos con los outliers

honeyBee tiene 2 outliers

bumbles tiene 3 outliers

andrena tiene 1 outliers

osmia tiene 6 outliers

fruitset tiene 9 outliers

fruitmass tiene 2 outliers

seeds tiene 2 outlier

In [464]:
df.head()

,clonesize,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield,honeybeeBC
0,37.5,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.410652,0.408159,31.678898,3813.165795,0.301481
1,37.5,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,1.0,0.10,0.444254,0.425458,33.449385,4947.605663,0.301481
2,37.5,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,16.0,0.26,0.383787,0.399172,30.546306,3866.798965,0.301481
3,37.5,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,1.0,0.10,0.407564,0.408789,31.562586,4303.943030,0.301481
4,37.5,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.354413,0.382703,28.873714,3436.493543,0.301481


In [465]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

print(IQR)
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df_out.shape

clonesize                 12.500000
bumbles                    0.130000
andrena                    0.250000
osmia                      0.250000
MaxOfUpperTRange          11.600000
MinOfUpperTRange           5.200000
AverageOfUpperTRange       7.200000
MaxOfLowerTRange          10.200000
MinOfLowerTRange           3.000000
AverageOfLowerTRange       5.000000
RainingDays               20.230000
AverageRainingDays         0.290000
fruitset                   0.106571
fruitmass                  0.059869
seeds                      6.123577
yield                   1897.334830
honeybeeBC                 0.083903
dtype: float64


(752, 17)

A continuacion haremos un scalado de las variable numericas


altos VIF - esto representa multicolinariedad

5       MaxOfUpperTRange           inf
6       MinOfUpperTRange           inf

8       MaxOfLowerTRange           inf
9       MinOfLowerTRange           inf

7   AverageOfUpperTRange 94,389,548.91
10  AverageOfLowerTRange 48,271,143.78
11           RainingDays         64.10
12    AverageRainingDays         71.32
13              fruitset         80.87
14             fruitmass        764.37
15                 seeds        566.71

In [466]:
from sklearn.preprocessing import StandardScaler

In [467]:
y = df_out['yield']

In [468]:
df_out.drop(columns=['yield'], axis=1, inplace=True)

C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\3510087700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.drop(columns=['yield'], axis=1, inplace=True)


In [469]:
X_Total = pd.DataFrame(StandardScaler().fit_transform(df_out), columns=df_out.columns)  

In [470]:
type(X_Total)

pandas.core.frame.DataFrame

In [471]:
df_out['AverageOfUpperTRangeSC'] = StandardScaler().fit_transform( np.array(df_out['AverageOfUpperTRange'])
                                                                  .reshape(-1, 1) )

df_out['AverageOfLowerTRangeSC'] = StandardScaler().fit_transform(np.array(df_out['AverageOfLowerTRange'])
                                                                  .reshape(-1,1) )

df_out['RainingDaysSC']  = StandardScaler().fit_transform(np.array(df_out['RainingDays'])
                                                                  .reshape(-1,1) )

df_out['AverageRainingDaysSC']  = StandardScaler().fit_transform(np.array(df_out['AverageRainingDays'])
                                                                  .reshape(-1,1) )

df_out['fruitsetSC']  = StandardScaler().fit_transform(np.array(df_out['fruitset'])
                                                                  .reshape(-1,1) )

df_out['fruitmassSC']   = StandardScaler().fit_transform(np.array(df_out['fruitmass'])
                                                                  .reshape(-1,1) )

df_out['seedsSC']   = StandardScaler().fit_transform(np.array(df_out['seeds'])
                                                                  .reshape(-1,1) )

C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\3388460586.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['AverageOfUpperTRangeSC'] = StandardScaler().fit_transform( np.array(df_out['AverageOfUpperTRange'])
C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\3388460586.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['AverageOfLowerTRangeSC'] = StandardScaler().fit_transform(np.array(df_out['AverageOfLowerTRange'])
C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\3388460586.py:7: Sett

In [472]:
df_out.head()

,clonesize,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,honeybeeBC,AverageOfUpperTRangeSC,AverageOfLowerTRangeSC,RainingDaysSC,AverageRainingDaysSC,fruitsetSC,fruitmassSC,seedsSC
0,37.5,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.410652,0.408159,31.678898,0.301481,0.430009,0.419327,-0.206039,-0.377097,-1.277828,-1.044250,-1.128742
1,37.5,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,1.0,0.10,0.444254,0.425458,33.449385,0.301481,0.430009,0.419327,-1.450217,-1.326308,-0.827835,-0.592303,-0.701814
2,37.5,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,16.0,0.26,0.383787,0.399172,30.546306,0.301481,1.348923,1.354981,-0.206039,-0.377097,-1.637605,-1.279035,-1.401850
3,37.5,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,1.0,0.10,0.407564,0.408789,31.562586,0.301481,1.348923,1.354981,-1.450217,-1.326308,-1.319181,-1.027781,-1.156789
4,37.5,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.354413,0.382703,28.873714,0.301481,0.430009,0.419327,0.457522,0.394137,-2.030966,-1.709321,-1.805172


In [473]:
df_out.drop(columns =['AverageOfUpperTRange'], axis = 1, inplace = True)
df_out.drop(columns =['AverageOfLowerTRange'], axis = 1, inplace = True)
df_out.drop(columns =['RainingDays'], axis = 1, inplace = True)
df_out.drop(columns =['AverageRainingDays'], axis = 1, inplace = True)
df_out.drop(columns =['fruitset'], axis = 1, inplace = True)
df_out.drop(columns =['fruitmass'], axis = 1, inplace = True)
df_out.drop(columns =['seeds'], axis = 1, inplace = True)

C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\2626722146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.drop(columns =['AverageOfUpperTRange'], axis = 1, inplace = True)
C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\2626722146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.drop(columns =['AverageOfLowerTRange'], axis = 1, inplace = True)
C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\2626722146.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [474]:
X_Total.to_csv('../data/processed/X_1.csv') 

In [475]:
df_out.to_csv('../data/processed/X_2.csv')

In [476]:
df_out.drop(columns=['MaxOfUpperTRange','MaxOfLowerTRange'], axis = 1, inplace = True)

C:\Users\laptop\AppData\Local\Temp\ipykernel_8300\4132881150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.drop(columns=['MaxOfUpperTRange','MaxOfLowerTRange'], axis = 1, inplace = True)


In [477]:
df_out.to_csv('../data/processed/X_3.csv')

In [478]:
y.to_csv('../data/processed/y.csv', index = False)